In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
#Checking for device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cuda


In [ ]:
#Transform for dataprocessing resize images, add augmintation technique to increase orignal image
#ToTensor change pixel color and change the range , matrix col rep RGB, rows rep mean and std deviation 
transformer=transforms.Compose([
                                transforms.Resize((150,150)),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), # 0 - 255 to 0 - 1 numpy to tensors
                                transforms.Normalize([0.5, 0.5, 0.5], 
                                [0.5, 0.5, 0.5]) # 0-1 to [-1, 1], formula (x-mean)/std
])

In [ ]:
#DataLoader
train_path='/content/drive/MyDrive/scene_detection/seg_train/seg_train'
test_path= '/content/drive/MyDrive/scene_detection/seg_test/seg_test'


train_loader = DataLoader(torchvision.datasets.ImageFolder(train_path, transform=transformer),
                          batch_size=256, shuffle=True)

test_loader = DataLoader(torchvision.datasets.ImageFolder(test_path, transform=transformer),
                          batch_size=32, shuffle=True)


In [ ]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [ ]:
#CNN Network Class

class ConvNet(nn.Module):
  def __init__(self, num_classes=6):
    super(ConvNet,self).__init__()

    #Output size after convolution filter
        #((w-f+2P)/s) +1

    ##Input shape= (256,3,150,150)
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)

    #shape= (256,12,150,150)
    self.bn1 = nn.BatchNorm2d(num_features=12)
    self.relu1 = nn.ReLU()

    self.pool = nn.MaxPool2d(kernel_size=2)
    #Reduce the image size be factor 2
    #Shape= (256,12,75,75)

    #Add second Convolution layer and relu layer shape is (256,20,75,75)

    self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
    self.relu2 = nn.ReLU()
    
    #Add third Convolution layer and relu layer shape is (256,32,75,75)

    self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.bn3 = nn.BatchNorm2d(num_features=32)
    self.relu3 = nn.ReLU()
    
    #Adding fully connected layer we feed  input feature which is d, h and image of C layer output
    self.fc = nn.Linear(in_features= 75 * 75 * 32, out_features = num_classes)

  #Feed forward function
  def forward(self, input):
    output = self.conv1(input)
    output = self.bn1(output)
    output = self.relu1(output)

    output = self.pool(output)

    output = self.conv2(output)
    output = self.relu2(output)

    output = self.conv3(output)
    output = self.bn3(output)
    output = self.relu3(output)

                #Above output will be in matrix form, with shape (256,32,75,75)
    #reshape function then feed inside
    output = output.view(-1,32 * 75 * 75 )
    output = self.fc(output)

    return output

In [ ]:
model = ConvNet(num_classes=6).to(device)

In [ ]:
#Calling Optimizer
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 10

In [ ]:
#claculating the size of traning and testing images using glob.glob func
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))

In [ ]:
print(train_count, test_count)

14034 3000


In [ ]:
#Model training and saving best model
#And calculate traing and testing loss for each epoch 

best_acc = 0.0 # will update inside loop for each epoc

for epoch in range(num_epochs):

  #evaluate traning and testing dataset
  model.train() # o train traning data 
  train_acc = 0.0
  train_loss = 0.0

  for i, (images, labels) in enumerate(train_loader):
    if torch.cuda.is_available():
      images = Variable(images.cuda())
      labels = Variable(labels.cuda())

    optimizer.zero_grad()

    output = model(images) #it give us prediction
    loss = loss_function(output,labels) #predicted and actual value the loss
    loss.backward()
    optimizer.step()#will updated the weight and bais

    train_loss += loss.cpu().data*images.size(0)
    _,prediction = torch.max(output.data,1)

    train_acc += int(torch.sum(prediction == labels.data))

  train_acc = train_acc/train_count
  train_loss = train_loss/train_count

  # Evaluation on testing dataset
  model.eval()

  test_acc = 0.0
  for i, (images,labels) in enumerate(test_loader):
      if torch.cuda.is_available():
          images=Variable(images.cuda())
          labels=Variable(labels.cuda())

      output = model(images)
      _,prediction=torch.max(output.data,1)
      test_acc += int(torch.sum(prediction == labels.data))

  test_acc = test_acc/test_count

  print('Epoch: '+str(epoch)+ 'Train Loss: '+str(train_loss)+ 'Train Accuracy: '+str(train_acc)+ 
        'Test Accuracy: '+str(test_acc))
      
  #save the best model
  if test_acc > best_acc:
    torch.save(model.state_dict(), 'best_checkpoint.model')
    best_acc = test_acc    

Epoch: 0Train Loss: tensor(1.0986)Train Accuracy: 0.7496793501496366Test Accuracy: 0.6186666666666667
Epoch: 1Train Loss: tensor(0.8670)Train Accuracy: 0.8043323357560211Test Accuracy: 0.7223333333333334
Epoch: 2Train Loss: tensor(0.5321)Train Accuracy: 0.8664671512042184Test Accuracy: 0.7236666666666667
Epoch: 3Train Loss: tensor(0.3954)Train Accuracy: 0.8954681487815306Test Accuracy: 0.7163333333333334
Epoch: 4Train Loss: tensor(0.2700)Train Accuracy: 0.9248966794926606Test Accuracy: 0.7
Epoch: 5Train Loss: tensor(0.2336)Train Accuracy: 0.9350149636596836Test Accuracy: 0.7536666666666667
Epoch: 6Train Loss: tensor(0.1807)Train Accuracy: 0.9474134245404019Test Accuracy: 0.756
Epoch: 7Train Loss: tensor(0.1010)Train Accuracy: 0.9692888698874162Test Accuracy: 0.7636666666666667
Epoch: 8Train Loss: tensor(0.1550)Train Accuracy: 0.9520450334900955Test Accuracy: 0.753
Epoch: 9Train Loss: tensor(0.0869)Train Accuracy: 0.9724241128687473Test Accuracy: 0.7746666666666666
